In [ ]:
def solve_knapsack_ga(weights, values, capacity,
                               population_size=200,
                               num_generations=1000,
                               mutation_rate=0.01,
                               tournament_size=5,
                               elitism=True,
                               penalty_coef=0.9):
    n = len(weights)

    # --- FITNESS WITH SOFT PENALTY ---
    def fitness(ind):
        w = (weights * ind).sum()
        v = (values  * ind).sum()
        if w <= capacity:
            return v
        else:
            # Penalize overweight
            return v - penalty_coef * (w - capacity)
    def create():
        return np.random.randint(0, 2, size=n)
    
    def tour(pop, fits):
        competitors = random.sample(range(len(pop)), tournament_size)
        return pop[max(competitors, key=lambda i:fits[i])].copy()
    
    def crossover(p1, p2):
        pt = random.randint(1, n-1)
        return np.concatenate([p1[:pt], p2[pt:]]), np.concatenate([p2[:pt], p1[pt:]])
     def mutate(ind):
        for i in range(n):
            if random.random() < mutation_rate:
                ind[i] ^= 1
        return ind
    pop = [create() for _ in range(population_size)]
    best_val, best_ind = 0, pop[0].copy()
    for _ in range(num_generations):
        fits = [fitness(ind) for ind in pop]
        idx = int(np.argmax(fits))
        if fits[idx] > best_val:
            best_val, best_ind = fits[idx], pop[idx].copy()
            new_pop = []
        if elitism:
            new_pop.append(best_ind.copy())
        while len(new_pop) < population_size:
            p1, p2 = tour(pop, fits), tour(pop, fits)
            c1, c2 = crossover(p1, p2)
            new_pop.extend([mutate(c1), mutate(c2)])
        pop = new_pop[:population_size]
    return best_val, best_ind